In [1]:
import numpy as np
import itertools
import copy

In [2]:
coin = [10300, 9600, 9800, 8200, 7800, 8300, 9500, 9800, 10200, 9500]

In [3]:
# coin 최대 이득 계산 C_list 함수
def max_revenue(C_list):
    return max([max(C_list[i+1:len(C_list)])-C_list[i] for i in range(len(C_list)-1)])

In [4]:
print(f'최대 이득은 {max_revenue(coin)}원 입니다.')

최대 이득은 2400원 입니다.


In [5]:
# 마을 크기 입력받기 rule 함수
def rule():
    while(1):
        try:
            row_num=int(input('마을의 크기를 입력하시오 :').strip())
            print('\n각 행의 원소를 띄어쓰기로 구분하여 입력하시오.')
            break
        except ValueError:
            print('\n정수만 입력하시오.\n')
    square=[]
    for row in range(1,row_num+1):
        i=1
        while(i):
            try:
                r=input(str(row)+'번째 행 :').strip().split(' ')
                assert len(r)==row_num
                square.append([int(j) for j in r])
                i=0
            except AssertionError:
                print('\n입력한 행의 길이가 '+str(row_num)+'이 아닙니다.\n다시 입력하시오.\n')
            except ValueError:
                print('\n정수만 입력하시오.\n')
    square=np.array(square)
    return(square)

In [6]:
# 최대값 계산 알고리즘 market 함수
def market(matrix):
    num_chestnut=copy.deepcopy(matrix)
    row_num=matrix.shape[0]
    for i in range(row_num):
        for j in range(row_num):
            if i==0 and j!=0:
                num_chestnut[i][j]+=num_chestnut[i][j-1]
            elif i!=0 and j==0:
                num_chestnut[i][j]+=num_chestnut[i-1][j]
            elif i>0 and j>0:
                num_chestnut[i][j]+=max(num_chestnut[i-1][j],num_chestnut[i][j-1])
    return(num_chestnut)

문제 설명에서 저는 조금 바꿔서 작성하였습니다.

난이도를 낮춘것은 아닙니다.

1.한 번에 여러 지역의 법을 수정할 수 있도록 한다.

2.하지만 큰 변화는 위험하므로 같은 지역에 여러 번 명령을 주어 2이상 변화가 생긴다면,
명령을 다시 하도록 한다.

In [7]:
def how_many(matrix):
    row_num=matrix.shape[0]
    square=matrix
    num_chestnut=market(square)
    print('\n현재 시장의 형태는 다음과 같습니다.')
    print(f'지역별 개수\n{square}\n지역별 최대값\n{num_chestnut}\n')
    print(f'시장에 유통되는 수량은 {sum(list(itertools.chain(*market(square))))}개 입니다.')
    command=[]
    tcommand=[]
    tmp_square=copy.deepcopy(square)
    while(1):
        whether=input('\n시장에 개입하시겠습니까? [Yes/No]\n').strip().lower()
        if whether=='yes' or whether=='y':
            print('\n명령을 입력하시오.\n각 명령은 \'A/B 행번호 열번호\'의 형태로 띄어쓰기로 구분하여 입력할 것.')
            print('명령 그만두기 : stop\n명령 취소하기 : cancel\n')
            i=1
            while(1):
                try:
                    tmp_command=input(str(i)+'번째 명령 : ').strip().lower().split(' ')
                    if tmp_command[0]=='stop':
                        command.append(list(itertools.chain(*tcommand)))
                        square=copy.deepcopy(tmp_square)
                        num_chestnut=market(square)
                        print('현재까지의 명령을 적용합니다.')
                        print('\n현재 시장의 형태는 다음과 같습니다.')
                        print(f'지역별 개수\n{square}\n지역별 최대값\n{num_chestnut}')
                        break
                    elif tmp_command[0]=='cancel':
                        tmp_square=copy.deepcopy(square)
                        tcommand=[]
                        print('모든 명령을 취소합니다.\n현재 시장의 형태는 다음과 같습니다.')
                        print(f'\n지역별 개수\n{square}\n지역별 최대값\n{num_chestnut}')
                        break
                    else:
                        tmp_command[1:]=[int(elm) for elm in tmp_command[1:]]
                        assert len(tmp_command)==3
                        assert max(tmp_command[1:])<=row_num and min(tmp_command[1:])>=1
                        assert tmp_command[0]=='a' or tmp_command[0]=='b'
                        try:
                            tmp=copy.deepcopy(tmp_square)
                            if tmp_command[0]=='a':
                                tmp_square[tmp_command[1]-1,tmp_command[2]-1]+=1
                            elif tmp_command[0]=='b':
                                tmp_square[tmp_command[1]-1,tmp_command[2]-1]-=1
                            assert max([abs(i) for i in list(itertools.chain(*(tmp_square-square)))])<=1
                            print(f'적용 시 유통되는 수량은 {sum(list(itertools.chain(*market(tmp_square))))}개 입니다.')
                            i+=1
                            tcommand.append(tmp_command)
                        except AssertionError:
                            print('\n이 명령을 수행하면 2 이상의 변화가 생깁니다.\n명령을 다시 입력하십시오.\n')
                            tmp_square=tmp
                except (AssertionError,TypeError,ValueError):
                    print('\n잘못된 명령입니다.\n다시 입력하시오.\n')
        elif whether=='no' or whether=='n':
            break
        else:
            print('\n잘못된 명령입니다.\n다시 입력하시오.\n')
    command=np.array(list(itertools.chain(*command))).reshape((-1,3))
    print(f'\n시장에 유통되는 수량은 {sum(list(itertools.chain(*market(square))))}개 이며, 지금까지 적용한 전체 명령은\n{command}\n입니다.')
    return 0

In [8]:
square=rule()

# 마을의 크기를 입력하시오 :3

# 각 행의 원소를 띄어쓰기로 구분하여 입력하시오.
# 1번째 행 :3 2 7
# 2번째 행 :4 2 6
# 3번째 행 :5 3 8

마을의 크기를 입력하시오 :3

각 행의 원소를 띄어쓰기로 구분하여 입력하시오.
1번째 행 :3 2 7
2번째 행 :4 2 6
3번째 행 :5 3 8


In [9]:
how_many(square)


현재 시장의 형태는 다음과 같습니다.
지역별 개수
[[3 2 7]
 [4 2 6]
 [5 3 8]]
지역별 최대값
[[ 3  5 12]
 [ 7  9 18]
 [12 15 26]]

시장에 유통되는 수량은 107개 입니다.

시장에 개입하시겠습니까? [Yes/No]
Yes

명령을 입력하시오.
각 명령은 'A/B 행번호 열번호'의 형태로 띄어쓰기로 구분하여 입력할 것.
명령 그만두기 : stop
명령 취소하기 : cancel

1번째 명령 : A 1 2
적용 시 유통되는 수량은 111개 입니다.
2번째 명령 : B 3 2
적용 시 유통되는 수량은 110개 입니다.
3번째 명령 : A 1 2

이 명령을 수행하면 2 이상의 변화가 생깁니다.
명령을 다시 입력하십시오.

3번째 명령 : A 1 a

잘못된 명령입니다.
다시 입력하시오.

3번째 명령 : stop
현재까지의 명령을 적용합니다.

현재 시장의 형태는 다음과 같습니다.
지역별 개수
[[3 3 7]
 [4 2 6]
 [5 2 8]]
지역별 최대값
[[ 3  6 13]
 [ 7  9 19]
 [12 14 27]]

시장에 개입하시겠습니까? [Yes/No]
y

명령을 입력하시오.
각 명령은 'A/B 행번호 열번호'의 형태로 띄어쓰기로 구분하여 입력할 것.
명령 그만두기 : stop
명령 취소하기 : cancel

1번째 명령 : B 3 3
적용 시 유통되는 수량은 109개 입니다.
2번째 명령 : B 1 2
적용 시 유통되는 수량은 105개 입니다.
3번째 명령 : cancel
모든 명령을 취소합니다.
현재 시장의 형태는 다음과 같습니다.

지역별 개수
[[3 3 7]
 [4 2 6]
 [5 2 8]]
지역별 최대값
[[ 3  6 13]
 [ 7  9 19]
 [12 14 27]]

시장에 개입하시겠습니까? [Yes/No]
yes

명령을 입력하시오.
각 명령은 'A/B 행번호 열번호'의 형태로 띄어쓰기로 구분하여 입력할 것.
명령 그만두기 : stop
명령 취소하기 : cancel

1번째 명령 : A

0